# Event Card Bonus
There are event cards that increase odds that an endeavor will be successful. For the "plus 1" event, major success count as two successes. Great! We'll need to account for two outcomes in each combination: major successes and blanks. The formula for computing these odds is a bit trickier since a major success will cancel out a blank roll. Therefore, a simulation will be used to sample the probability space and be used as a sanity check to verify the formula.

In [ ]:
%matplotlib inline

import itertools
import matplotlib
import matplotlib.pyplot
import numpy
import pandas
import scipy.misc
import scipy.special
import scipy.stats
import seaborn

# The simulation


In [ ]:
p_blank = 1.0/3
p_major = 1.0/6

def simulate_roll(total_dice):
    x = numpy.random.uniform(size=total_dice)
    conds = [x <= p_blank, (x > p_blank) & (x <= p_blank + p_major), (x > p_blank + p_major)]
    # each blank counts as 1, a major success counts as -1, and a regular success counts as 0
    roll_array = numpy.piecewise(x, conds, [1, -1, 0])
    return numpy.sum(roll_array)

def blank_roll_simulator(total_dice, n):
    sim_list = [simulate_roll(total_dice) for i in range(n)]
    return numpy.array(sim_list)
    #return {"num_blank" : num_blank_array, "p_blank" : p_blank_array}

In [ ]:
n = 10000
blank_count_list = [blank_roll_simulator(i, n) for i in numpy.arange(1,16)]
for i, array in enumerate(blank_count_list):
    array[array < 0] = 0
    blank_count_list[i] = array
blank_count_itemfreq = [scipy.stats.itemfreq(a) for a in blank_count_list]

In [ ]:
# sdo_sim will give the probability of a specific instance occuring.
# the rows are the size of the dice pool
# the columns are the specific number of blanks after accounting for major successes
sdo_sim = numpy.zeros((15,16))
for i, array in enumerate(blank_count_itemfreq):
    for j in array:
        sdo_sim[i, j[0].astype(int)] = j[1]/n
        


In [ ]:
row_blank = numpy.arange(1,16) # number of dice showing the blank face
col_dice = numpy.arange(0,16) # number of dice in the pool
sdo_sim_dataframe = pandas.DataFrame(data = sdo_sim, index = row_blank, columns = col_dice)
sdo_sim_dataframe_stack = sdo_sim_dataframe.stack().reset_index().rename(columns = {"level_0" : "number_blank", "level_1" : "number_dice_pool", 0 : "probability_number_blank"})
sdo_sim_mask = numpy.triu(numpy.ones((15,16)), 2)
sdo_sim_pivot = sdo_sim_dataframe_stack.pivot("number_blank", "number_dice_pool", "probability_number_blank")

In [ ]:
seaborn.set(style = "white")
seaborn.set_context("poster")
matplotlib.pyplot.figure(figsize=(24, 18))
cmap = seaborn.cubehelix_palette(n_colors = 6,
                                start = 1.5,
                                rot = 1.5,
                                gamma = 1.5,
                                hue = 1.0,
                                dark = 0.525,
                                light = 0.96,
                                reverse = False,
                                as_cmap = True)
ax = seaborn.heatmap(sdo_sim_pivot, 
                     annot = True, 
                     cmap = cmap,
                     cbar = False,
                     mask = sdo_sim_mask,
                     annot_kws = {"weight" : "extra bold"},
                     fmt = ".1%", 
                     linewidths = 2,
                     vmin=0.0, 
                     vmax=1.0)

ax.set_title("Ship Damage Probabilty for Event Card Bonus (plus 1, simulation):\n'Probability of exactly X many blanks given Y sized dice pool'",
            fontsize=32)
ax.set_xlabel("Number of blanks", fontsize=32)
ax.set_ylabel("Size of dice pool", fontsize=32)


fig = ax.get_figure()
fig.savefig("sdo_sim_15_plus1.pdf")

In [ ]:
# for the greater than or equal probabilites, sum along the rows
sdo_sim_gte = numpy.zeros((15,15))

for (i, j), _ in numpy.ndenumerate(sdo_sim_gte):
    sdo_sim_gte[i, j] = numpy.sum(sdo_sim[i, j+1:])


In [ ]:
row_blank = numpy.arange(1,16) # number of dice showing the blank face
col_dice = numpy.arange(1,16) # number of dice in the pool
sdo_sim_gte_dataframe = pandas.DataFrame(data = sdo_sim_gte, index = row_blank, columns = col_dice)
sdo_sim_gte_dataframe_stack = sdo_sim_gte_dataframe.stack().reset_index().rename(columns = {"level_0" : "number_blank", "level_1" : "number_dice_pool", 0 : "probability_GTE_to_number_blank"})
sdo_sim_gte_mask = numpy.triu(numpy.ones((15,15)), 1)
sdo_sim_gte_pivot = sdo_sim_gte_dataframe_stack.pivot("number_blank", "number_dice_pool", "probability_GTE_to_number_blank")

In [ ]:
seaborn.set(style = "white")
seaborn.set_context("poster")
matplotlib.pyplot.figure(figsize=(24, 18))
cmap = seaborn.cubehelix_palette(n_colors = 6,
                                start = 1.5,
                                rot = 1.5,
                                gamma = 1.5,
                                hue = 1.0,
                                dark = 0.525,
                                light = 0.96,
                                reverse = False,
                                as_cmap = True)
ax = seaborn.heatmap(sdo_sim_gte_pivot, 
                     annot = True, 
                     cmap = cmap,
                     cbar = False,
                     mask = sdo_sim_gte_mask,
                     annot_kws = {"weight" : "extra bold"},
                     fmt = ".1%", 
                     linewidths = 2,
                     vmin=0.0, 
                     vmax=1.0)

ax.set_title("Ship Damage Probabilty for Event Card Bonus (plus 1, simulation):\n'Probability at least X many blanks given Y sized dice pool'",
            fontsize=32)
ax.set_xlabel("Number of blanks", fontsize=32)
ax.set_ylabel("Size of dice pool", fontsize=32)


fig = ax.get_figure()
fig.savefig("sdo_gte_sim_15_plus1.pdf")

# The formula
The odds for major successes a $1/6$. The key to calculating the number of outcomes is the [Multinomial Coefficient](https://en.wikipedia.org/wiki/Multinomial_theorem). 

In [ ]:
p_blank = 1.0/3
p_major = 1.0/6

def prob_blank_and_major(num_blank, num_major, total_dice):
    if num_blank + num_major > total_dice:
        p = 0.0
    else:    
        # https://en.wikipedia.org/wiki/Multiset#Counting_multisets
        p = (p_blank)**num_blank * \
        (p_major)**num_major * \
        (1-p_blank-p_major)**(total_dice-num_blank-num_major) * \
        scipy.misc.factorial(total_dice) / \
        (scipy.misc.factorial(num_blank) * \
         scipy.misc.factorial(num_major) * \
         scipy.misc.factorial(total_dice - num_blank - num_major)) 
    return p

def prob_blank(num_blank, total_dice):
    if total_dice < num_blank:
        p = 0.0
    else:
        rng_blank = numpy.arange(num_blank, total_dice + 1)
        array_blank_and_major = [prob_blank_and_major(i, i-num_blank, total_dice) for i in numpy.arange(num_blank, 1 + numpy.floor((total_dice + num_blank)/2))]
        p = numpy.sum(array_blank_and_major)
    return p 

def prob_blank_or_worse(num_blank, total_dice):
    if total_dice < num_blank:
        p = 0.0
    else:
        rng_blank = numpy.arange(num_blank, total_dice + 1)
        array_blank = [prob_blank(i, total_dice) for i in rng_blank]
        p = numpy.sum(array_blank)
    return p

In [ ]:
row_blank = numpy.arange(1,16) # number of dice showing the blank face
col_dice = numpy.arange(1,16) # number of dice in the pool
sdo = numpy.zeros((numpy.size(row_blank),numpy.size(col_dice)))
prob = [prob_blank_or_worse(*i) for i in itertools.product(row_blank, col_dice)]
ind = [tuple(numpy.subtract(i,1)) for i in itertools.product(row_blank, col_dice)]
for idx, val in enumerate(ind):
    sdo[val] = prob[idx]
sdo_dataframe = pandas.DataFrame(data = sdo, index = row_blank, columns = col_dice)
sdo_dataframe_stack = sdo_dataframe.stack().reset_index().rename(columns = {"level_0" : "number_blank", "level_1" : "number_dice_pool", 0 : "probability_GTE_to_number_blank"})
sdo_mask = numpy.triu(numpy.ones((15,15)), 1)
sdo_pivot = sdo_dataframe_stack.pivot("number_dice_pool", "number_blank", "probability_GTE_to_number_blank")

In [ ]:
seaborn.set(style = "white")
seaborn.set_context("poster")
matplotlib.pyplot.figure(figsize=(24, 18))
cmap = seaborn.cubehelix_palette(n_colors = 6,
                                start = 1.5,
                                rot = 1.5,
                                gamma = 1.5,
                                hue = 1.0,
                                dark = 0.525,
                                light = 0.96,
                                reverse = False,
                                as_cmap = True)
ax = seaborn.heatmap(sdo_pivot, 
                     annot = True, 
                     cmap = cmap,
                     cbar = False,
                     mask = sdo_mask,
                     annot_kws = {"weight" : "extra bold"},
                     fmt = ".1%", 
                     linewidths = 2,
                     vmin=0.0, 
                     vmax=1.0)

ax.set_title("Ship Damage Probabilty for Event Card Bonus (plus 1):\n'Probability at least X many blanks given Y sized dice pool'",
            fontsize=32)
ax.set_xlabel("Number of blanks", fontsize=32)
ax.set_ylabel("Size of dice pool", fontsize=32)


fig = ax.get_figure()
fig.savefig("sdo_15_plus1.pdf")

In [ ]:
sdo_dataframe_7 = sdo_dataframe_stack.loc[(sdo_dataframe_stack["number_blank"] < 8) & (sdo_dataframe_stack["number_dice_pool"] < 8)]
sdo_mask_7 = numpy.triu(numpy.ones((7,7)), 1)
sdo_pivot_7 = sdo_dataframe_7.pivot("number_dice_pool", "number_blank", "probability_GTE_to_number_blank")

ax2 = seaborn.heatmap(sdo_pivot_7, 
                     annot = True, 
                     cmap = cmap,
                     cbar = False,
                     mask = sdo_mask_7,
                     annot_kws = {"weight" : "extra bold"},
                     fmt = ".1%", 
                     linewidths = 1.5,
                     vmin=0.0, 
                     vmax=1.0)

ax2.set_title("Ship Damage Probabilty:\n'Probability at least X many blanks given Y sized dice pool', or\n'Probability of greater than or equal to the number of blanks for a given dice pool'",
            fontsize=20)
ax2.set_xlabel("Number of blanks", fontsize=24)
ax2.set_ylabel("Size of dice pool", fontsize=24)

fig = ax2.get_figure()
fig.savefig("sdo_7_plus1.pdf")

In [ ]:
p_major = 1.0/6

def prob_major(num_major, total_dice):
    if total_dice < num_major:
        p = 0.0
    else:
        p = (p_major)**num_major * \
        (1-p_major)**(total_dice-num_major) * \
        scipy.misc.comb(total_dice, total_dice - num_major)
    return p

def prob_major_or_better(num_major, total_dice):
    if total_dice < num_major:
        p = 0.0
    else:
        rng_major = numpy.arange(num_major, total_dice + 1)
        array_major = [prob_major(n, total_dice) for n in rng_major]
        p = numpy.sum(array_major)
    return p

In [ ]:
row_major = numpy.arange(1,16) # number of dice showing the major success face
col_dice = numpy.arange(1,16) # number of dice in the pool
sdo_major = numpy.zeros((numpy.size(row_major),numpy.size(col_dice)))
prob = [prob_major_or_better(*i) for i in itertools.product(row_major, col_dice)]
ind = [tuple(numpy.subtract(i,1)) for i in itertools.product(row_major, col_dice)]
for idx, val in enumerate(ind):
    sdo_major[val] = prob[idx]
sdo_major_dataframe = pandas.DataFrame(data = sdo_major, index = row_major, columns = col_dice)
sdo_major_dataframe_stack = sdo_major_dataframe.stack().reset_index().rename(columns = {"level_0" : "number_major", "level_1" : "number_dice_pool", 0 : "probability_GTE_to_number_major"})
sdo_major_mask = numpy.triu(numpy.ones((15,15)), 1)
sdo_major_pivot = sdo_major_dataframe_stack.pivot("number_dice_pool", "number_major", "probability_GTE_to_number_major")

In [ ]:
seaborn.set(style = "white")
seaborn.set_context("poster")
matplotlib.pyplot.figure(figsize=(24, 18))
cmap = seaborn.cubehelix_palette(n_colors = 6,
                                start = 1.5,
                                rot = 1.5,
                                gamma = 1.5,
                                hue = 1.0,
                                dark = 0.525,
                                light = 0.96,
                                reverse = False,
                                as_cmap = True)
ax = seaborn.heatmap(sdo_major_pivot, 
                     annot = True, 
                     cmap = cmap,
                     cbar = False,
                     mask = sdo_mask,
                     annot_kws = {"weight" : "extra bold"},
                     fmt = ".1%", 
                     linewidths = 2,
                     vmin=0.0, 
                     vmax=1.0)

ax.set_title("Ship Damage Probabilty for Event Card Bonus (plus 1):\n'Probability at least X many major successes given Y sized dice pool'",
            fontsize=32)
ax.set_xlabel("Number of major successes", fontsize=32)
ax.set_ylabel("Size of dice pool", fontsize=32)


fig = ax.get_figure()
fig.savefig("sdo_15_major_successes.pdf")